In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Simulating real-time localization

In this notebook we will be simulating the real-time capabilities of the deep learning model trained to predict the source location in terms of a 3D Gaussian-distribution. To this end we pick time-windows from the continuous data around the times we know there was an event according to the earthquake catalog. We pick a time-window slightly before arrival of the event until the event passed and simulate a real-time processing scenario by predicting the output based on the seismic data.

In [10]:
# import libraries

import tensorflow as tf

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import glob
import os

from obspy.signal.filter import bandpass
import obspy

import copy

In [3]:
! pip install obspy

We need to define a class to read in the data and apply some pre-processing steps

In [4]:
class FieldDataReader:
    
    def __init__(self):
        self.filename = "filename"
        self.data = np.zeros((1,1))
        self.df = 2
        
    def get_data(self, filename, receiver_file):
        
        data, starttime = self.read_segy(filename)
        data = self.remove_traces(receiver_file)
        data = self.bpf()
        
        return data, starttime
        

    def read_segy(self, filename):
        
        if os.path.isfile(filename) ==  False:
            raise ValueError("File does not exist")
            
        stream = obspy.read(filename)
        self.data = np.stack(t.data for t in stream.traces)
        self.data = np.transpose(self.data)
        
        self.df = stream.traces[0].stats.sampling_rate
        
        return self.data, stream.traces[0].stats.starttime
    
    def remove_traces(self, receiver_file):
        
        '''
        remove_traces: removes all traces that are not used by the U-Net
        The shallow traces are ignored and furthermore a set of fixed traces are removed
        '''
        
        if os.path.isfile(receiver_file) ==  False:
            raise ValueError("File does not exist")
            
        receiver_info = pd.read_csv(receiver_file)
        group = receiver_info['Group'].values
        id_kill = np.where(group==4)[0]

        # Keep only deepest receivers
        id_kill = np.where(group==2)[0] # id of shallow receivers 1
        id_kill = np.append(id_kill, np.where(group==3)[0]) # id of shallow receivers 2
        id_kill = np.append(id_kill, np.where(group==4)[0]) # id of permantetely dead traces and duplicates
        id_dead = np.array([0, 1, 2, 4, 5, 8, 16, 19, 22, 25, 31, 32, 33, 34, 35, 36, 37, 42, 43, 45, 46, 47, 51, 52, 55, 56, 57, 59, 60, 62, 63, 66, 67, 68, 69, 71, 72, 73, 75, 76, 79, 83, 84, 85, 86, 88, 89, 92, 93, 94, 96, 97, 98, 100, 103, 106, 107, 108, 110, 111, 112, 113, 114, 116, 119, 122, 123, 127, 128, 129, 131, 132, 134, 136, 137, 139, 142, 144, 145, 146, 150, 152, 155, 156, 157, 158, 160, 165, 166, 167, 169, 170, 172, 174, 175, 178, 179, 180, 181, 182, 186, 187, 189, 190, 193])
        
        self.data = np.delete(self.data, id_kill, axis=1)
        self.data = np.delete(self.data, id_dead, axis=1)
        
        return self.data
    
    def bpf(self):
        # Band-pass filter the data
        self.data = np.insert(self.data, 0, np.zeros((100, 97)), axis=0)
        self.data = np.append(self.data, np.zeros((100,97)),axis=0)
        
        for j in range(0, self.data.shape[1]):
            self.data[:,j] = bandpass(self.data[:,j], 5., 50., df=self.df, corners=12, zerophase=True)
                
        self.data = self.data[100:self.data.shape[0]-100,:96]
        
        return self.data

    def bandpassfilter(self, c1=5, c2=50):
        # Band-pass filter the data
        self.data = np.insert(self.data, 0, np.zeros((100, data.shape[1])), axis=0)
        self.data = np.append(self.data, np.zeros((100,data.shape[1])),axis=0)
        
        for j in range(0, self.data.shape[1]):
            self.data[:,j] = bandpass(self.data[:,j], c1, c2, df=self.df, corners=12, zerophase=True)
                
        self.data = self.data[100:self.data.shape[0]-100,:]
        
        return self.data
    

Define a function to create wiggle plots

In [48]:
def wiggle(
    DataO: np.ndarray,
    x=None,
    t=None,
    ax=None,
    skipt=1,
    lwidth=.5,
    gain=1,
    typeD='VA',
    color='red',
    perc=100):

    """
    wiggle(DataO, x=None, t=None, maxval=-1, skipt=1, lwidth=.5, gain=1, typeD='VA', color='red', perc=100)

    This function generates a wiggle plot of the seismic data.

    Parameters
    ----------
    DataO: np.ndarray of shape (# time samples, # traces)
        Seismic data

    Optional parameters
    -------------------
    x: np.ndarray of shape Data.shape[1]
        x-coordinates to Plot
    t: np.ndarray of shape Data.shap[0]
        t-axis to plot
    skipt: int
        Skip trace, skips every n-th trace
    ldwidth: float
        line width of the traces in the figure, increase or decreases the traces width
    typeD: string
        With or without filling positive amplitudes. Use type=None for no filling
    color: string
        Color of the traces
    perc: float
        nth parcintile to be clipped

    Returns
    -------
    Seismic wiggle plot

    Adapted from segypy (Thomas Mejer Hansen, https://github.com/cultpenguin/segypy/blob/master/segypy/segypy.py)


    """
    # Make a copy of the original, so that it won't change the original one ouside the scope of the function
    Data = copy.copy(DataO)

    # calculate value of nth-percentile, when perc = 100, data won't be clipped.
    nth_percentile = np.abs(np.percentile(Data, perc))

    # clip data to the value of nth-percentile
    Data = np.clip(Data, a_min=-nth_percentile, a_max = nth_percentile)

    ns = Data.shape[0]
    ntraces = Data.shape[1]

    #fig = plt.gca()
    ax = ax or plt.gca()
    ntmax=1e+9 # used to be optinal

    if ntmax<ntraces:
        skipt=int(np.floor(ntraces/ntmax))
        if skipt<1:
                skipt=1

    if x is not None:
        x=x
        ax.set_xlabel('Distance [m]')
    else:
        x=range(0, ntraces)
        ax.set_xlabel('Trace number')

    if t is not None:
        t=t
        yl='Time [s]'
    else:
        t=np.arange(0, ns)
        yl='Sample number'

    dx = x[1]-x[0]

    Dmax = np.nanmax(Data)
    maxval = np.abs(Dmax)

    for i in range(0, ntraces, skipt):

       # use copy to avoid truncating the data
        trace = copy.copy(Data[:, i])
        trace = Data[:, i]
        trace[0] = 0
        trace[-1] = 0
        traceplt = x[i] + gain * skipt * dx * trace / maxval
        traceplt = np.clip(traceplt, a_min=x[i]-dx, a_max=(dx+x[i]))

        ax.plot(traceplt, t, color=color, linewidth=lwidth)

        offset = x[i]

        if typeD=='VA':
            for a in range(len(trace)):
                if (trace[a] < 0):
                    trace[a] = 0
            ax.fill_betweenx(t, offset, traceplt, where=(traceplt>offset), interpolate='True', linewidth=0, color=color)
            ax.grid(False)

        ax.set_xlim([x[0]-1, x[-1]+1])

    ax.invert_yaxis()
    ax.set_ylim([np.max(t), np.min(t)])
    ax.set_ylabel(yl)

    return ax

Load the localization model

In [5]:
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
    return dice

def dice_coef_gaussian(y_true, y_pred, smooth=1):
    y_true = tf.dtypes.cast(y_true>0.1, tf.int32)
    y_true = tf.dtypes.cast(y_true, tf.float32)
    y_pred = tf.dtypes.cast(y_pred>0.1, tf.int32)
    y_pred = tf.dtypes.cast(y_pred, tf.float32)
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
    return dice

path_model = 'drive/My Drive/Texas_TL/TL_models/TL_TFR_post/QNet_pre_test_long2_97.h5'
model = tf.keras.models.load_model(
        path_model,
        custom_objects={
            'ReLU':tf.keras.layers.ReLU, 'dice_coef': dice_coef,
            'dice_coef_gaussian': dice_coef_gaussian,
        })

Define file paths to data, catalog, etc

In [6]:
ls drive/My\ Drive/Texas_TL/Data/continuousData/

Catalog10.08.19.csv                     June_19_00-03/
Catalog10.08.19_refinedloc2.csv         Masters.csv
Catalog10.08.19_relative_locations.csv  rec_ordered.csv
Catalog_refinedloc2.csv


In [7]:
catalog = 'drive/My Drive/Texas_TL/Data/continuousData/Catalog10.08.19_refinedloc2.csv'
path_cont_data = 'drive/My Drive/Texas_TL/Data/continuousData/June_19_00-03'
receiver_file = 'drive/My Drive/Texas_TL/Data/continuousData/rec_ordered.csv'

# Make sure to use catalog with origin_time in the right format: hour:minute:seconds
df_cat = pd.read_csv(catalog)
df_cat.head(3)

# Write dataframe with only events from June 19 2010 and with origin time 00-03
df = df_cat[df_cat['Event_Date']=='19-Jun-2010']
df = df[df['Origin_Time']<'04:00:00.000']
df.head(3)

,Event_Number,Event_Number_In_File,File_ID,File_Number,Event_Date,Origin_Time,Origin_Time_Relative_To_File,Detection_Time_Relative_To_File,X,Y,Z,X_error,Y_error,Z_error,X_abs,Y_abs,Z_abs,STA_LTA,SNR,Semblance,Effective_Number_Of_Receivers,Magnitude,Source_Moment,VOL,CLVD,DC,P1_Strike,P1_Dip,P1_Rake,P2_Strike,P2_Dip,P2_Rake,SMTensor_NN,SMTensor_EE,SMTensor_ZZ,SMTensor_NE,SMTensor_NZ,SMTensor_EZ,P1_Strike_err,P2_Strike_err,P2_Dip_err,P1_Dip_err,P1_Rake_err,P2_Rake_err,KGN_err
346,351,1,c1006190002_1_dspk,367,19-Jun-2010,00:02:05.800,0.207648,0.912,7640.867882,5493.048401,1940.995065,11.050916,12.380326,17.427732,608901.1272,214485.1070,2136.193562,9.432876,0.933994,0.525577,102,1.095130,5.529618e+10,-0.316526,-0.138059,0.545415,41.725240,48.746625,91.841408,218.933896,41.287098,87.901826,-3.089027e+10,-3.753943e+10,1.609807e+10,2.032276e+10,3.563374e+09,-3.064343e+09,1.221749,1.753589,0.573430,0.577527,1.033521,1.187078,0.765500
347,352,1,c1006190002_2_dspk,368,19-Jun-2010,00:02:19.089,0.255096,0.860,7334.460447,4968.107968,2062.446338,13.284441,12.080841,5.706975,608631.7489,213945.1182,1748.677304,10.781706,1.001056,0.559866,116,1.048934,4.714120e+10,-0.284632,-0.337521,0.377847,7.464690,49.909693,74.717706,210.455461,42.440540,107.386007,-9.012573e+09,-4.268295e+10,1.138026e+10,1.273106e+10,-2.125293e+09,-4.808787e+09,2.248624,1.809068,1.021727,0.598089,2.279953,2.296744,1.555135
348,353,1,c1006190005_1_dspk,369,19-Jun-2010,00:05:31.366,0.286063,0.924,7420.278749,5141.521508,2079.194701,14.328995,15.242810,12.834264,608719.6094,214104.7689,1819.709823,17.504139,1.632269,0.631352,115,1.391807,1.540676e+11,-0.336748,-0.249849,0.413403,208.196692,49.487161,93.312953,23.104605,40.624774,86.130842,-5.571058e+10,-1.313549e+11,3.008283e+10,4.753035e+10,-8.810028e+09,1.171848e+10,1.568781,1.551765,0.448923,0.504193,1.339322,1.597286,0.946724


Define functions to process data for prediciton

In [166]:

def interp_time(data):
  dt = 0.002
  t = np.arange(0,1401)*dt
  tn = np.linspace(0,t[-1],1024)
  dtn = tn[1]-tn[0]
  x_new = np.zeros((len(tn),data.shape[1]))
  for i in range(x_new.shape[1]):
    x_new[:,i] = np.interp(tn, t, data[:,i])

  return x_new

def interp_time2(tv):
  dt = 0.002
  t = np.arange(0,1401)*dt
  tn = np.linspace(0,t[-1],1024)
  dtn = tn[1]-tn[0]
  tv = np.interp(tn, t, tv)

  return tv

def normalize(data):
    data = data/np.max(np.abs(data))
    return data

Write loop to go read in continuous data, pick windows aroung events (given by catalog) and make prediction using localization model

In [168]:
xs = np.linspace(5500,8500,128)
ys = np.linspace(3500,6100,96)
zs = np.linspace(1000,3200,64)

for i in range(3,4):
 
  fldr = path_cont_data + '/hour_0{}/'.format(i)
    
  df_tmp = df[df['Origin_Time']>'0{}:00:00.0'.format(i)]
  df_tmp = df_tmp[df_tmp['Origin_Time']<'0{}:00:00.0'.format(i+1)]

  for j, t_event in enumerate(df_tmp['Origin_Time'].values):

    idx = df_tmp.index[df_tmp['Origin_Time'] == t_event].to_list()[0]

    # Retrieve location of the event in the catalog
    x = int(df_tmp.loc[idx].X)
    y = int(df_tmp.loc[idx].Y)
    z = int(df_tmp.loc[idx].Z)

    # Retrieve files containing event (each file corresponds to one minute of data)
    fn = fldr + '1006190{}{}.sgy'.format(str(i), t_event.split(':')[1])
    reader = FieldDataReader()
    data, tt = reader.get_data(fn, receiver_file)

    # Select start and end time around events
    sec_start = int(t_event.split(':')[-1].split('.')[0])
    idx1 = int(sec_start/0.002)-int(3/0.002)
    idx2 = int(sec_start/0.002)+int(6/0.002)

    st=sec_start-3
    et=sec_start+6
    time_vec = np.arange(st, et, 0.002)

    datas = data[idx1:idx2,:]

    tc = 0

    for c in range(11):

      tv = interp_time2(time_vec[tc:tc+1401])

      # Prepare data for prediction
      data_sub = datas[tc:tc+1401,:]
      data_sub = interp_time(data_sub)
      data_sub = normalize(data_sub)
      data_sub = np.reshape(data_sub, (1, data_sub.shape[0], data_sub.shape[1],1))

      tc+=300

      # Make prediction     
      prediction = model.predict(data_sub)[0,:]

      maxIdx = np.unravel_index(np.argmax(prediction), prediction.shape)

      fig = plt.figure(constrained_layout=True, figsize=(10,6))
      gs = GridSpec(2, 2, figure=fig)
      ax1 = fig.add_subplot(gs[0, 0])
      # identical to ax1 = plt.subplot(gs.new_subplotspec((0, 0), colspan=3))
      ax2 = fig.add_subplot(gs[1, 0])
      ax3 = fig.add_subplot(gs[:, 1])

      im1 = ax1.pcolormesh(xs,ys,prediction[:,:,maxIdx[2]].T, cmap=plt.cm.jet, vmin=0, vmax=1)
      ax1.set_xlabel('Easting (m)', fontsize=12)
      ax1.set_ylabel('Northing (m)', fontsize=12)
      ax1.scatter(x, y, marker='*', color='white',s=250)
      ax1.set_xlim([5700, 8300])
      ax1.set_ylim([3700, 5900])
      im2 = ax2.pcolormesh(xs,zs,prediction[:,maxIdx[1],:].T, cmap=plt.cm.jet, vmin=0.0, vmax=1)
      ax2.set_xlabel('Easting (m)', fontsize=12)
      ax2.set_ylabel('Depth (m)', fontsize=12)
      ax2.scatter(x, z, marker='*', color='white',s=250)
      ax2.set_xlim([5700, 8300])
      ax2.set_ylim([1000, 3200])
      ax2.invert_yaxis()
      #cbar_ax = fig.add_axes([0.25, 1.05, 0.55, 0.05])
      wiggle(data_sub[0,:,:,0], perc=97, color='k', t=tv, ax=ax3)
      ax3.set_xlabel('Traces')
      ax3.set_ylabel('Time')

      #fig.colorbar(im1, cax=cbar_ax, orientation="horizontal")
      plt.tight_layout()
      plt.savefig('drive/My Drive/Texas_TL/Figures/contPredictions/hour_0{}_min_{}_inp_{}.png'.format(i,t_event.split(':')[1], c))
      plt.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
/usr/local/lib/python3.7/dist-pack

ValueError: ignored